In [33]:
import torch
from torchvision import models
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
from torchvision import transforms
from PIL import Image

Getting our CNN ready to predict

In [34]:
alexnet = models.alexnet(pretrained=True)
alexnet.eval()

with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]

### Creating our Custom Dataset

CSV we have

In [35]:
st=pd.read_csv('datasetinfo.csv',header=None)
st

,0
0,dog.jpg
1,cat.jpg


In [40]:
class ImageClassificationDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        self.datasetinfo = pd.read_csv(csv_file,header=None)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.datasetinfo)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.datasetinfo.iloc[idx, 0])
        print(img_name)
        image = Image.open(img_name)
        image.show()
        if self.transform:
            sample = self.transform(image)
        item = torch.unsqueeze(sample, 0)

        return item

In [41]:
transformed_dataset = ImageClassificationDataset(csv_file='datasetinfo.csv',
                                           root_dir='pics/',
                                           transform=transforms.Compose([            #[1]
                                                         transforms.Resize(256),                    #[2]
                                                         transforms.CenterCrop(224),                #[3]
                                                         transforms.ToTensor(),                     #[4]
                                                         transforms.Normalize(                      #[5]
                                                         mean=[0.485, 0.456, 0.406],                #[6]
                                                         std=[0.229, 0.224, 0.225]                  #[7]
                                                         )])
                                          )

In [42]:
print(transformed_dataset[0])

pics/dog.jpg
tensor([[[[ 0.6734,  0.6049,  0.5364,  ...,  0.7933,  0.6906,  0.6734],
          [ 0.6221,  0.5878,  0.5878,  ...,  0.6906,  0.6734,  0.7077],
          [ 0.6221,  0.6734,  0.6734,  ...,  0.5878,  0.6221,  0.6734],
          ...,
          [ 0.1426,  0.1426,  0.0227,  ..., -0.3198,  0.0398,  0.0912],
          [-0.5082, -0.1486, -0.1828,  ...,  0.0227, -0.0458,  0.3138],
          [-0.4568, -0.3027, -0.2342,  ...,  0.1083,  0.1254,  0.2624]],

         [[ 1.0455,  0.9755,  0.9055,  ...,  1.1856,  1.0455,  0.9930],
          [ 1.0105,  0.9580,  0.9580,  ...,  1.0805,  1.0280,  1.0280],
          [ 1.0105,  1.0455,  1.0455,  ...,  0.9755,  0.9755,  0.9930],
          ...,
          [ 0.5553,  0.6078,  0.5553,  ...,  0.0126,  0.3978,  0.4678],
          [ 0.0126,  0.4328,  0.4503,  ...,  0.3803,  0.3102,  0.6604],
          [ 0.1702,  0.3627,  0.4853,  ...,  0.4503,  0.4328,  0.5378]],

         [[ 0.2871,  0.2522,  0.1999,  ...,  0.3393,  0.2522,  0.2348],
          [ 0.199

In [39]:
out = alexnet(transformed_dataset[0])

_, indices = torch.sort(out, descending=True)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
[(classes[idx], percentage[idx].item()) for idx in indices[0][:5]]

pics/dog.jpg


[('Labrador retriever', 41.58518600463867),
 ('golden retriever', 16.59165382385254),
 ('Saluki, gazelle hound', 16.286855697631836),
 ('whippet', 2.853914976119995),
 ('Ibizan hound, Ibizan Podenco', 2.3924756050109863)]